<a href="https://colab.research.google.com/github/Yasser1910/Trade-Oman/blob/main/oman_export_products_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import io, requests, certifi
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

pd.set_option("display.float_format", lambda x: f"{x:,.0f}")


In [10]:
import pandas as pd


url = "https://raw.githubusercontent.com/Yasser1910/Trade-Oman/refs/heads/main/oman_export_products.csv"



df = pd.read_csv("https://raw.githubusercontent.com/Yasser1910/Trade-Oman/refs/heads/main/oman_export_products.csv")

print("✅ Loaded from GitHub:", df.shape)
df.head(5)


✅ Loaded from GitHub: (99127, 12)


,year,trade_type,hs_code,hs_name,value_omr,date,hs_code_full,hs_full_int,section_code_2,section_name,section_name_clean,bec_broad
0,1998,Export,"1,012,110",Purebred Horses Of Arab breed,"20,300",1998,10121100,10121100,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
1,1998,Export,"1,019,000",Live Mules And Hinnies,7,1998,10190000,10190000,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
2,1998,Export,"1,022,100",PureBred Cattle For Breeding,"19,350",1998,10221000,10221000,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
3,1998,Export,"1,029,000",Live Bovine Animals Excl Cattle And Buffalo,"16,970",1998,10290000,10290000,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
4,1998,Export,"1,042,010",Live PureBred Breeding Goats,"1,220,112",1998,10420100,10420100,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages


In [11]:
# normalize column names
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# common columns expected: year, trade_type, hs_code, hs_name, value_omr
# (plus: section_name_clean, bec_broad if present)
df["year"] = pd.to_numeric(df.get("year"), errors="coerce").astype("Int64")
df["trade_type"] = df.get("trade_type").astype(str).str.title()
df["hs_code"] = df.get("hs_code").astype(str).str.strip()
df["value_omr"] = pd.to_numeric(df.get("value_omr"), errors="coerce").fillna(0.0)

# oil flag: HS 27 = Oil
df["oil_flag"] = np.where(df["hs_code"].str.startswith("27"), "Oil (HS 27)", "Non-Oil")

print("✅ Cleaned. Columns:", list(df.columns))
df.sample(5)


✅ Cleaned. Columns: ['year', 'trade_type', 'hs_code', 'hs_name', 'value_omr', 'date', 'hs_code_full', 'hs_full_int', 'section_code_2', 'section_name', 'section_name_clean', 'bec_broad', 'oil_flag']


,year,trade_type,hs_code,hs_name,value_omr,date,hs_code_full,hs_full_int,section_code_2,section_name,section_name_clean,bec_broad,oil_flag
1850,1999,Re-Export,20079919.0,Jam Jellies Marmalade Of Fruits Excluding Peac...,"34,019",1999,20079919,20079919,4,PREPARED FOODSTUFFS,Prepared foods; beverages; tobacco,Consumer Goods,Non-Oil
13893,2006,Re-Export,84369900.0,Parts Of Agricultural Horticultural Forestry O...,750,2006,84369900,84369900,16,MACHINERY AND MECHANICAL APPLIANCES,"Machinery, electrical equipment & parts",Capital Goods,Non-Oil
68952,2020,Export,72121000.0,FlatRolled Products Of Iron Or NonAlloy Steel ...,51,2020,72121000,72121000,15,BASE METALS AND ARTICLES OF BASE METAL,Base metals & articles,Industrial Supplies,Non-Oil
86969,2023,Export,15180090.0,Other Of Animal or vegetable fats and oils and...,"18,095",2023,15180090,15180090,3,ANIMAL OR VEGETABLE FATS AND OILS,Animal/vegetable fats & oils,Industrial Supplies,Non-Oil
57307,2018,Re-Export,17019911.0,Refined Crystal Sugar Not Flavoured Nor Colour...,"529,707",2018,17019911,17019911,4,PREPARED FOODSTUFFS,Prepared foods; beverages; tobacco,Consumer Goods,Non-Oil


In [13]:
# Total Exports = Export + Re-Export (imports excluded)
exports_plus_re = df[df["trade_type"].isin(["Export", "Re-Export"])].copy()
total = (exports_plus_re.groupby("year", as_index=False)["value_omr"]
         .sum().rename(columns={"value_omr":"Total_Exports_All"}))

# Non-Oil (Export only)
non_oil = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non-Oil")]
           .groupby("year", as_index=False)["value_omr"].sum()
           .rename(columns={"value_omr":"NonOil_Exports"}))

# Oil (Export only)
oil = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Oil (HS 27)")]
       .groupby("year", as_index=False)["value_omr"].sum()
       .rename(columns={"value_omr":"Oil_Exports"}))

# merge & compute shares/YoY
series = (total.merge(non_oil, on="year", how="outer")
               .merge(oil, on="year", how="outer")
               .fillna(0.0).sort_values("year"))

series["NonOil_Share_pct"] = np.where(series["Total_Exports_All"]>0,
                                      series["NonOil_Exports"]/series["Total_Exports_All"]*100, np.nan)
series["YoY_Total_%"]  = series["Total_Exports_All"].pct_change()*100
series["YoY_NonOil_%"] = series["NonOil_Exports"].pct_change()*100
series["YoY_Oil_%"]    = series["Oil_Exports"].pct_change()*100

series.head(10)


,year,Total_Exports_All,NonOil_Exports,Oil_Exports,NonOil_Share_pct,YoY_Total_%,YoY_NonOil_%,YoY_Oil_%
0,1998,"2,071,803,925","186,394,040","1,392,154,980",9,NaN,NaN,NaN
1,1999,"2,727,083,011","187,295,006","2,084,443,561",7,32,0,50
2,2000,"4,281,139,167","233,000,980","3,549,505,207",5,57,24,70
3,2001,"4,229,515,282","251,264,287","3,400,639,684",6,-1,8,-4
4,2002,"4,257,236,204","253,496,451","3,277,036,168",6,1,1,-4
5,2003,"4,425,279,368","298,931,539","3,525,578,278",7,4,18,8
6,2004,"5,083,561,968","407,065,064","4,138,208,017",8,15,36,17
7,2005,"7,114,460,435","539,218,830","5,991,334,390",8,40,32,45
8,2006,"8,370,404,461","648,567,870","6,954,891,285",8,18,20,16
9,2007,"9,157,238,607","912,965,640","7,240,778,668",10,9,41,4


In [14]:
# 4.1 Total vs Non-Oil (levels)
fig1 = go.Figure()
fig1.add_scatter(x=series["year"], y=series["Total_Exports_All"], mode="lines+markers",
                 name="Total Exports (Export + Re-Export)", line=dict(width=3))
fig1.add_scatter(x=series["year"], y=series["NonOil_Exports"], mode="lines+markers",
                 name="Non-Oil Exports (Export only)", line=dict(width=3))
fig1.update_layout(title="Total vs Non-Oil Exports (Imports excluded from total)",
                   xaxis_title="Year", yaxis_title="OMR", template="plotly_white")
fig1.show()

# 4.2 Non-oil share of total
fig2 = px.line(series, x="year", y="NonOil_Share_pct", markers=True,
               title="Non-Oil Exports as % of Total Exports (Export + Re-Export)")
fig2.update_traces(line=dict(width=3), marker=dict(size=8))
fig2.update_layout(template="plotly_white", yaxis_title="Share (%)")
fig2.show()

# 4.3 YoY growth
fig3 = go.Figure()
for col, name in [("YoY_Total_%","Total"),("YoY_NonOil_%","Non-Oil"),("YoY_Oil_%","Oil")]:
    fig3.add_scatter(x=series["year"], y=series[col], mode="lines+markers", name=name)
fig3.add_hline(y=0, line_dash="dot", line_color="gray")
fig3.update_layout(title="YoY Growth (Totals exclude Imports)",
                   xaxis_title="Year", yaxis_title="YoY Growth (%)", template="plotly_white")
fig3.show()

# 4.4 product counts (Export only)
exp_only = df[df["trade_type"]=="Export"]
counts = (exp_only[exp_only["value_omr"]>0]
          .groupby(["year","oil_flag"])["hs_code"].nunique()
          .reset_index())
fig4 = px.line(counts, x="year", y="hs_code", color="oil_flag", markers=True,
               title="Number of Products Exported (Export only)")
fig4.update_traces(line=dict(width=3))
fig4.update_layout(template="plotly_white", yaxis_title="Unique HS Codes")
fig4.show()


In [15]:
# Non-Oil by BEC (Export only)
non_oil_bec = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non-Oil")]
               .groupby(["year","bec_broad"], as_index=False)["value_omr"].sum())

# Total by BEC = Export + Re-Export (imports excluded)
total_bec = (df[df["trade_type"].isin(["Export","Re-Export"])]
             .groupby(["year","bec_broad"], as_index=False)["value_omr"].sum())

def stacked_area(df_bec, title):
    wide = df_bec.pivot(index="year", columns="bec_broad", values="value_omr").fillna(0).sort_index()
    fig = go.Figure()
    for col in wide.columns:
        fig.add_scatter(x=wide.index, y=wide[col], stackgroup="one", name=col)
    fig.update_layout(title=title, xaxis_title="Year", yaxis_title="OMR", template="plotly_white")
    fig.show()

stacked_area(non_oil_bec, "Non-Oil Exports by BEC (Export only)")
stacked_area(total_bec,   "Total Exports by BEC (Export + Re-Export; Imports EXCLUDED)")


In [16]:
latest_year = int(df["year"].dropna().max()); prev_year = latest_year - 1
non_oil_export = df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non-Oil")].copy()

cur = (non_oil_export[non_oil_export["year"]==latest_year]
       .groupby(["hs_code","hs_name"], as_index=False)["value_omr"].sum()
       .rename(columns={"value_omr":"value_curr"}))

prev = (non_oil_export[non_oil_export["year"]==prev_year]
        .groupby(["hs_code","hs_name"], as_index=False)["value_omr"].sum()
        .rename(columns={"value_omr":"value_prev"}))

prod = cur.merge(prev, on=["hs_code","hs_name"], how="left").fillna({"value_prev":0.0})
prod["abs_delta"] = prod["value_curr"] - prod["value_prev"]
prod["yoy_pct"] = np.where(prod["value_prev"]>0, prod["abs_delta"]/prod["value_prev"]*100, np.nan)

top_val  = prod.sort_values("value_curr", ascending=False).head(10)
top_grow = prod.sort_values("abs_delta", ascending=False).head(10)

top_val[["hs_code","hs_name","value_curr","value_prev","abs_delta","yoy_pct"]]


,hs_code,hs_name,value_curr,value_prev,abs_delta,yoy_pct
273,26011200.0,Agglomerated Iron Ores And Concentrates Excl R...,"674,724,865","635,262,553","39,462,312",6
345,31021000.0,Urea Whether Or Not In Aqueous Solution Excl T...,"421,099,044","418,104,969","2,994,075",1
1029,72142040.0,Other bars and rods of iron or nonalloy steel ...,"249,612,049","223,470,552","26,141,497",12
308,29051100.0,Methanol Methyl Alcohol,"236,365,241","198,210,177","38,155,064",19
1193,76011000.0,Aluminium Not Alloyed Unwrought,"222,067,770","249,100,878","-27,033,108",-11
501,39076100.0,poly ethylene terephthalate Having a viscosity...,"204,902,697","161,365,837","43,536,860",27
535,39206200.0,Plates Sheets Film Foil And Strip Of NonCellul...,"172,083,069","157,979,433","14,103,636",9
999,72071100.0,SemiFinished Products Of Iron Or NonAlloy Stee...,"167,791,621","120,271,006","47,520,615",40
488,39021000.0,Polypropylene In Primary Forms,"163,900,363","174,344,707","-10,444,344",-6
484,39012000.0,Polyethylene With A Specific Gravity Of 094 In...,"149,311,837","102,391,061","46,920,776",46


In [18]:
no_sec = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non-Oil")]
          .groupby(["year","section_name_clean"], as_index=False)["value_omr"].sum())
no_tot = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non-Oil")]
          .groupby("year", as_index=False)["value_omr"].sum()
          .rename(columns={"value_omr":"non_oil_total"}))

pivot_sec = no_sec.pivot(index="year", columns="section_name_clean", values="value_omr").fillna(0.0).sort_index()
pivot_sec = pivot_sec.merge(no_tot.set_index("year"), left_index=True, right_index=True, how="left")

pivot_sec["total_prev"] = pivot_sec["non_oil_total"].shift(1)
deltas = pivot_sec.drop(columns=["non_oil_total","total_prev"]).diff()
contrib_pp_sec = deltas.div(pivot_sec["total_prev"], axis=0) * 100

contrib_long = contrib_pp_sec.reset_index().melt(id_vars="year", var_name="HS Section", value_name="pp")
fig = px.bar(contrib_long, x="year", y="pp", color="HS Section",
             title="Contribution to Non-Oil Exports YoY Growth by HS Section (pp)",
             labels={"pp":"Percentage points"})
fig.update_layout(barmode="relative", template="plotly_white")
fig.show()
